To use pytorch Dataset and Dataloader 
1. Dataset need to implement the __len__() and __getitem__ function 
2. Dataloader, specify the batch size and shuffle 
3. training loop 
4. loss function 
5. prediction 

torch.utils.data.TensorDataset is a convenient class in PyTorch that allows you to create a dataset from one or more tensors. It is useful when your data is already in the form of tensors and you want to quickly create a dataset without writing a custom Dataset class.

In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
train = pd.read_csv('../data/sensor/train.csv')
labels = pd.read_csv('../data/sensor/train_labels.csv')
test = pd.read_csv('../data/sensor/test.csv')

In [7]:
print(train.head())

   sequence  subject  step  sensor_00  sensor_01  sensor_02  sensor_03  \
0         0       47     0  -0.196291   0.112395        1.0   0.329204   
1         0       47     1  -0.447450   0.134454        1.0  -0.658407   
2         0       47     2   0.326893  -0.694328        1.0   0.330088   
3         0       47     3   0.523184   0.751050        1.0   0.976991   
4         0       47     4   0.272025   1.074580        1.0  -0.136283   

   sensor_04  sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
0  -1.004660  -0.131638  -0.127505   0.368702       -0.1  -0.963873   
1   0.162495   0.340314  -0.209472  -0.867176        0.2  -0.301301   
2   0.473678   1.280479  -0.094718   0.535878        1.4   1.002168   
3  -0.563287  -0.720269   0.793260   0.951145       -0.3  -0.995665   
4   0.398579   0.044877   0.560109  -0.541985       -0.9   1.055636   

   sensor_10  sensor_11  sensor_12  
0  -0.985069   0.531893   4.751492  
1   0.082733  -0.231481   0.454390  
2   0.449221  -0.

In [8]:
from sklearn.preprocessing import StandardScaler

features = train.columns.tolist()[3:]
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.transform(test[features])
print(features)

['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12']


In [4]:
Window = 60

train = train.drop(["sequence", "subject", "step"], axis=1).to_numpy()
train = train.reshape(-1, train.shape[-1], Window)
train = np.transpose(train, (0, 2, 1))

y_train = labels['state'].to_numpy()

test = test.drop(["sequence", "subject", "step"], axis=1).to_numpy()
test = test.reshape(-1, test.shape[-1], Window)
test = np.transpose(test, (0, 2, 1))

In [5]:
print(train.shape)
print(y_train.shape)

(25968, 60, 13)
(25968,)


In [6]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset

print(torch.cuda.is_available())

train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.2, random_state=42)

train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
print(train_y[0:5])
train_tensor = TensorDataset(train_X, train_y)

valid_X = torch.tensor(valid_X, dtype=torch.float32)
valid_y = torch.tensor(valid_y, dtype=torch.float32) 
valid_tensor = TensorDataset(valid_X, valid_y)

test_X = torch.tensor(test, dtype=torch.float32)
test_tensor = TensorDataset(test_X)

train_loader = DataLoader(dataset=train_tensor, batch_size=64, shuffle=True)
valid_loader = DataLoader(dataset=valid_tensor, batch_size=64)
test_loader = DataLoader(dataset=test_tensor, batch_size=64)

True
tensor([0., 1., 0., 0., 1.])


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
%matplotlib inline

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

train = pd.read_csv('../data/sensor/train.csv')
test = pd.read_csv('../data/sensor/test.csv')
train_labels = pd.read_csv("../data/sensor/train_labels.csv")

train = train.set_index(["sequence", "subject", "step"])
test = test.set_index(["sequence", "subject", "step"])

def add_features(df, features):
    for feature in features:
        df_grouped = df.groupby("sequence")[feature]
        df_rolling = df_grouped.rolling(5, center=True)
        
        df[feature + "_lag1"] = df_grouped.shift(1)
        df[feature + "_diff1"] = df[feature] - df[feature + "_lag1"]
        df[feature + "_lag2"] = df_grouped.shift(2)
        df[feature + "_diff2"] = df[feature] - df[feature + "_lag2"]
        df[feature + "_roll_mean"] = df_rolling.mean().reset_index(0, drop=True)
        df[feature + "_roll_std"] = df_rolling.std().reset_index(0, drop=True)
    df.dropna(axis=0, inplace=True)
    return

features = ["sensor_{:02d}".format(i) for i in range(13)]
# add_features(train, features)
# add_features(test, features)
train.head()

input_size = train.shape[1]
sequence_length = len(train.index.get_level_values(2).unique())

# Scaling test and train
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Reshaping:
train = train.reshape(-1, sequence_length, input_size)
test = test.reshape(-1, sequence_length, input_size)
print("After Reshape")
print("Shape of training set: {}".format(train.shape))
print("Shape of test set: {}".format(test.shape))


# Splitting train data set into train and validation sets
# validation size is selected as 0.2
t_X, v_X, t_y, v_y = train_test_split(train, train_labels.state, test_size=0.20,
                                      shuffle=True, random_state=0)

# Converting train, validation and test data into tensors
train_X_tensor = torch.tensor(t_X).float()
val_X_tensor = torch.tensor(v_X).float()
test_tensor = torch.tensor(test).float()

# Converting train and validation labels into tensors
train_y_tensor = torch.tensor(t_y.values)
val_y_tensor = torch.tensor(v_y.values)

# Creating train and validation tensors
train_tensor = TensorDataset(train_X_tensor, train_y_tensor)
val_tensor = TensorDataset(val_X_tensor, val_y_tensor)

# Defining the dataloaders
dataloaders = dict()
dataloaders["train"] = DataLoader(train_tensor, batch_size=64, shuffle=True)
dataloaders["val"] = DataLoader(val_tensor, batch_size=32)
dataloaders["test"] = DataLoader(test_tensor, batch_size=32)
print("Dataloaders are created!")

After Reshape
Shape of training set: (25968, 60, 13)
Shape of test set: (12218, 60, 13)
Dataloaders are created!


In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, seq_len, dropout=0.5, output_size=1):
        super(RNN, self).__init__()
        
        # LSTM Layers
        self.lstm_1 = nn.LSTM(input_size, hidden_sizes[0], num_layers=2,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_21 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_22 = nn.LSTM(input_size, hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_31 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_32 = nn.LSTM(4*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_41 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_42 = nn.LSTM(4*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        hidd = 2*hidden_sizes[0] + 4*(hidden_sizes[1]+hidden_sizes[2]+hidden_sizes[3])
        self.lstm_5 = nn.LSTM(hidd, hidden_sizes[4], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Sequential(nn.Linear(2*hidden_sizes[4]*seq_len, 4096),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(4096, 1024),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(1024, output_size),
                                nn.Sigmoid()
                               )
        
    def forward(self, x):
        # lstm layers:
        x1, _ = self.lstm_1(x)
        
        x_x1, _ = self.lstm_21(x1)
        x_x2, _ = self.lstm_22(x)
        x2 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_31(x_x1)
        x_x2, _ = self.lstm_32(x2)
        x3 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_41(x_x1)
        x_x2, _ = self.lstm_42(x3)
        x4 = torch.cat([x_x1, x_x2], dim=2)
        x = torch.cat([x1, x2, x3, x4], dim=2)
        x, _ = self.lstm_5(x)
        
        # fully connected layers:
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [18]:
def training_loop(model, num_epochs, criterion, optimizer, train_loader, valid_loader, device):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device).unsqueeze(1).float()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")
        # running_loss = 0.0
        # with torch.no_grad():
        #     correct = 0
        #     total = 0
        #     for i, data in enumerate(valid_loader, 0):
        #         v_inputs, v_labels = data
        #         v_inputs = v_inputs.to(device)
        #         v_labels = v_labels.to(device).unsqueeze(1).float()
        #         outputs = model(v_inputs)
        #         loss = criterion(outputs, v_labels)
        #         running_loss += loss.item()
        #         predicts = torch.round(outputs).squeeze()
        #         total += v_labels.size(0)
        #         correct += (predicts == v_labels).sum().item()

        # print(f"Epoch {epoch+1}, Validation Loss: {running_loss/len(valid_loader)}")
        # print(f'Validation Accuracy: {100 * correct / total:.2f}%')oss: {running_loss/len(train_loader)}")
        
        # model.eval()
        # running_loss = 0.0
        # with torch.no_grad():
        #     correct = 0
        #     total = 0
        #     for i, data in enumerate(valid_loader, 0):
        #         v_inputs, v_labels = data
        #         v_inputs = v_inputs.to(device)
        #         v_labels = v_labels.to(device).unsqueeze(1).float()
        #         outputs = model(v_inputs)
        #         loss = criterion(outputs, v_labels)
        #         running_loss += loss.item()
        #         predicts = torch.round(outputs).squeeze()
        #         total += v_labels.size(0)
        #         correct += (predicts == v_labels).sum().item()

        # print(f"Epoch {epoch+1}, Validation Loss: {running_loss/len(valid_loader)}")
        # print(f'Validation Accuracy: {100 * correct / total:.2f}%')

In [8]:
### VALIDATION FUNCTION
def validation(model, loader, criterion, device="cpu"):
    model.eval()
    loss = 0
    preds_all = torch.LongTensor()
    labels_all = torch.LongTensor()
    
    with torch.no_grad():
        for batch_x, labels in loader:
            labels_all = torch.cat((labels_all, labels), dim=0)
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            output = model.forward(batch_x)
            loss += criterion(output,labels).item()
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
    total_loss = loss/len(loader)
    auc_score = roc_auc_score(labels_all, preds_all)
    return total_loss, auc_score

In [12]:

num_epochs = 30
# learning_rate = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# model = TimeSeriesClassifier(
#     input_size=13,
#     num_classes=1,
#     seq_len=60,
#     conv_filters=64,
#     lstm_units=128,
#     dropout_rate=0.2
# ).to(device)

# model = SimpleRNN(
#     input_size=13,
#     hidden_size=128,
#     seq_len=60
# ).to(device)

hidden_sizes = [288, 192, 144, 96, 32]
max_learning_rate = 0.001

model = RNN(
    input_size=13,
    hidden_sizes=hidden_sizes,
    seq_len=60
)
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=max_learning_rate)

for e in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_x, labels in dataloaders["train"]:
        batch_x, labels = batch_x.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()
        
        # Training 
        optimizer.zero_grad()
        output = model.forward(batch_x)
        loss = criterion(output, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_loss, train_auc = validation(model, dataloaders["train"], criterion, device)
    print(train_loss)
    # train_loss = 0.0
    # for i, (inputs, labels) in enumerate(dataloaders["train"]):
    #     inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1).float()
    #     optimizer.zero_grad()
    #     outputs = model(inputs)
    #     loss = criterion(outputs, labels)
    #     train_loss += loss.item()
    #     loss.backward()
    #     optimizer.step()
    # print(f"Epoch {e+1}, Loss: {train_loss / len(dataloaders['train'])}")

# training_loop(model, num_epochs, criterion, optimizer, dataloaders["train"], dataloaders["val"], device)

cuda
0.6936855932382436
0.6929685115814209


KeyboardInterrupt: 